<a href="https://colab.research.google.com/github/julianovale/PO240_Meta_heuristica/blob/main/PO240_Semana06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# colab: incluir !pip install
import numpy as np # módulo para manipulação de vetores e matrizes
from ortools.linear_solver import pywraplp  # CBC
import docplex.mp.sdetails                  # CPLEX
from docplex.mp.model import Model          # CPLEX
from ortools.sat.python import cp_model     # CPSAT

def leitura(arquivo):
    arq = open(arquivo, "r") # r- read
    lixo = arq.readline()  # linha com texto Number of jobs
    linha = arq.readline() # linha com número de tarefas
    N = int(linha)         # linha com o número de tarefas
    p = np.zeros(N, dtype=np.int64)  # p = [p0, p1,..., p(N-1)]
    w = np.zeros(N, dtype=np.int64)
    d = np.zeros(N, dtype=np.int64)
    lixo = arq.readline()  # linha com texto Job data (job number, release date, processing time, due date)
    for i in range(N):  # repetir N vezes
        # i-ésima linha
        linha = arq.readline().split()  # posições: 0-indice 1-p, 2-w, 3-d
        p[i] = int(linha[1])
        w[i] = int(linha[2])
        d[i] = int(linha[3])
    return N, p, w, d   

def modelo_Min_TerminoPonderado_CPLEX(N, p, w, d):
    # cria o modelo
    mdl = Model(name="Única Máquina")

    M = 2*sum(p)

    # alocação de memória para as estruturas do problema
    x = [0]*N  
    y = []
    for i in range(N):
        crialinha = [0]*N
        y.append(crialinha)

    # declaração das variáveis de decisão  
    for i in range(N):     # min, max 
        x[i] = mdl.integer_var(0, None, 'x'+str(i))  

    for i in range(N):
        for k in range(N):
            if k > i :            
                y[i][k] = mdl.binary_var('y'+str(i)+","+str(k))

    # declaração das restrições 
    for i in range(N):
        mdl.add_constraint( x[i] + p[i] <= d[i] )
        for k in range(N):
            if k > i :
                mdl.add_constraint( x[i] + p[i] <= x[k] + M*(1-y[i][k]) )
                mdl.add_constraint( x[k] + p[k] <= x[i] + M*y[i][k] )

    # função objetivo
    mdl.minimize( mdl.sum(w[i]*(x[i] + p[i]) for i in range(N)) )
    
    # limitar tempo de execução (em segundos)
    mdl.set_time_limit(1*60*60) 
    
    #resolver o problema
    mdl.solve()
    status = mdl.solve_details.status
    print("Status: ", status)
        
    print("Número de Variáveis: ", mdl.number_of_variables)  
    print("Número de Restrições: ", mdl.number_of_constraints)
    print("Tempo de execução (s): %.4f" %mdl.solve_details.time)
    print("Nós B&B: ", mdl.solve_details.nb_nodes_processed)
        
    print("Função-objetivo: ", mdl.objective_value)
    print("Best Bound: %.4f" %mdl.solve_details.best_bound)            
    print("GAP: %.4f" %mdl.solve_details.mip_relative_gap)   
    
    print("Lista de Alocação (tarefa, início):")
    aux = []        
    for i in range(N):
        aux.append((i, x[i].solution_value))        
    solOrd = sorted(aux, key=lambda tup: tup[1])
    print(solOrd)
    print()
    
def modelo_Min_TerminoPonderado_CBC(N, p, w, d):
    # cria o modelo
    mdl = pywraplp.Solver('Única Máquina', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

    M = 2*sum(p)

    # alocação de memória para as estruturas do problema
    x = [0]*N  
    y = []
    for i in range(N):
        crialinha = [0]*N
        y.append(crialinha)

    # declaração das variáveis de decisão    
    for i in range(N):     # min, max 
        x[i] = mdl.IntVar(0, mdl.infinity(), 'x'+str(i))  

    for i in range(N):
        for k in range(N):
            if k > i :            
                y[i][k] = mdl.IntVar(0, 1,'y'+str(i)+","+str(k))

    # declaração das restrições 
    for i in range(N):
        mdl.Add( x[i] + p[i] <= d[i] )
        for k in range(N):
            if k > i :
                mdl.Add( x[i] + p[i] <= x[k] + M*(1-y[i][k]) )
                mdl.Add( x[k] + p[k] <= x[i] + M*y[i][k] )

    # função objetivo
    mdl.Minimize( mdl.Sum(w[i]*(x[i] + p[i]) for i in range(N)) )
    
    # limitar tempo de execução (em milisegundos)
    mdl.SetTimeLimit(1*60*60*1000)   # 1 hora
    
    #resolver o problema
    status = mdl.Solve()
    print("Status: ", status)

    print("Número de Variáveis: ", mdl.NumVariables())  
    print("Número de Restrições: ", mdl.NumConstraints())
    print("Tempo de execução (s): %.4f" %float(mdl.WallTime()/1000))
    print("Nós B&B: ", mdl.nodes())
    
    print("Função-objetivo: ", mdl.Objective().Value())
    print("Best Bound: %.4f" %mdl.Objective().BestBound())
    
    gap = -1
    if mdl.Objective().Value():
        gap = (mdl.Objective().Value() - mdl.Objective().BestBound())/mdl.Objective().Value()
    print("GAP: %.4f" %gap)
    
    print("Lista de Alocação (tarefa, início):")
    aux = []
    for i in range(N):
        aux.append((i, x[i].solution_value()))
    solOrd = sorted(aux, key=lambda tup: tup[1])
    print(solOrd)
    print()
    
def modelo_Min_TerminoPonderado_CPSAT(N, p, w, d):
    # cria o modelo
    mdl = cp_model.CpModel()

    M = int(2*sum(p))

    # alocação de memória para as estruturas do problema
    x = [0]*N  
    y = []
    for i in range(N):
        crialinha = [0]*N
        y.append(crialinha)

    xmax = int(sum(p))
    # declaração das variáveis de decisão    
    for i in range(N):     # min, max 
        x[i] = mdl.NewIntVar(0, xmax, 'x'+str(i))  

    for i in range(N):
        for k in range(N):
            if k > i :            
                y[i][k] = mdl.NewBoolVar('y'+str(i)+","+str(k))

    # declaração das restrições 
    for i in range(N):
        mdl.Add( x[i] + p[i] <= d[i] )
        for k in range(N):
            if k > i :
                mdl.Add( x[i] + p[i] <= x[k] + M*(1-y[i][k]) )
                mdl.Add( x[k] + p[k] <= x[i] + M*y[i][k] )

    # função objetivo
    mdl.Minimize( sum(w[i]*(x[i] + p[i]) for i in range(N)) )
    
    # resolver com CP-SAT
    solver = cp_model.CpSolver()
    # núcleos para processamento
    solver.parameters.num_search_workers = 8
    # limitar tempo de execução (em segundos)
    solver.parameters.max_time_in_seconds = 1*60*60
    status = solver.Solve(mdl)
    status = solver.StatusName(status)
    print("Status: ", status)
    
    z = solver.ObjectiveValue()
    bb = solver.BestObjectiveBound()
    nos = solver.NumBranches()
    gap = -1
    if solver.ObjectiveValue():
        gap = (solver.ObjectiveValue() - solver.BestObjectiveBound())/solver.ObjectiveValue()
    print("Função-objetivo: ", z)
    print("Best Bound: %.4f" %bb)
    print("Nós B&B: ", nos)
    print("GAP: %.4f" %gap)
    print(solver.ResponseStats())    
    
    print("Lista de Alocação (tarefa, início):")
    aux = []
    for i in range(N):
        aux.append((i, solver.Value(x[i])))
    solOrd = sorted(aux, key=lambda tup: tup[1])
    print(solOrd)
    print()

In [ ]:
# *********************** 10 TAREFAS CPLEX ***************************
# Leitura do arquivo com os dados
# N (qtde tarefas), p (proc), w (peso), d (prazo) são as saídas da função leitura
N, p, w, d = leitura("instancia10.txt")  

print("===== CPLEX =====")
modelo_Min_TerminoPonderado_CPLEX(N, p, w, d)

===== CPLEX =====
Status:  integer optimal solution
Número de Variáveis:  55
Número de Restrições:  100
Tempo de execução (s): 0.2551
Nós B&B:  0
Função-objetivo:  17481
Best Bound: 17481.0000
GAP: 0.0000
Lista de Alocação (tarefa, início):
[(2, 0), (7, 79.0), (9, 133.0), (3, 198.0), (0, 253.0), (5, 306.0), (6, 379.0), (4, 507.0), (1, 644.0), (8, 785.0)]



In [ ]:
# *********************** 10 TAREFAS CBC ***************************
# Leitura do arquivo com os dados
# N (qtde tarefas), p (proc), w (peso), d (prazo) são as saídas da função leitura
N, p, w, d = leitura("instancia10.txt")  

print("===== CBC =====")
modelo_Min_TerminoPonderado_CBC(N, p, w, d)

===== CBC =====
Status:  0
Número de Variáveis:  55
Número de Restrições:  100
Tempo de execução (s): 3.5570
Nós B&B:  219
Função-objetivo:  17481.0
Best Bound: 17481.0000
GAP: 0.0000
Lista de Alocação (tarefa, início):
[(2, 0.0), (7, 79.0), (9, 133.0), (3, 198.0), (0, 253.0), (5, 306.0), (6, 379.0), (4, 507.0), (1, 644.0), (8, 785.0)]



In [ ]:
# *********************** 10 TAREFAS CP-SAT ***************************
# Leitura do arquivo com os dados
# N (qtde tarefas), p (proc), w (peso), d (prazo) são as saídas da função leitura
N, p, w, d = leitura("instancia10.txt")  

print("===== CP-SAT =====")
modelo_Min_TerminoPonderado_CPSAT(N, p, w, d)

===== CP-SAT =====
Status:  OPTIMAL
Função-objetivo:  17481.0
Best Bound: 17481.0000
Nós B&B:  201
GAP: 0.0000
CpSolverResponse:
status: OPTIMAL
objective: 17481
best_bound: 17481
booleans: 47
conflicts: 72
branches: 201
propagations: 1157
integer_propagations: 2568
walltime: 0.0393403
usertime: 0.0393404
deterministic_time: 0.00103972
primal_integral: 0.00260867

Lista de Alocação (tarefa, início):
[(2, 0), (7, 79), (9, 133), (3, 198), (0, 253), (5, 306), (6, 379), (4, 507), (1, 644), (8, 785)]



In [ ]:
# *********************** 15 TAREFAS CPLEX ***************************
# Leitura do arquivo com os dados
# N (qtde tarefas), p (proc), w (peso), d (prazo) são as saídas da função leitura
N, p, w, d = leitura("instancia15.txt")  

print("===== CPLEX =====")
modelo_Min_TerminoPonderado_CPLEX(N, p, w, d)

===== CPLEX =====
Status:  integer optimal solution
Número de Variáveis:  120
Número de Restrições:  225
Tempo de execução (s): 1.1708
Nós B&B:  4374
Função-objetivo:  57657
Best Bound: 57657.0000
GAP: 0.0000
Lista de Alocação (tarefa, início):
[(0, 0), (12, 53.0), (4, 106.0), (10, 243.0), (3, 390.0), (5, 445.0), (1, 518.0), (6, 659.0), (2, 787.0), (7, 866.0), (9, 920.0), (13, 985.0), (8, 1035.0), (11, 1162.0), (14, 1244.0)]



In [ ]:
# *********************** 15 TAREFAS CBC ***************************
# Leitura do arquivo com os dados
# N (qtde tarefas), p (proc), w (peso), d (prazo) são as saídas da função leitura
N, p, w, d = leitura("instancia15.txt")  

print("===== CBC =====")
modelo_Min_TerminoPonderado_CBC(N, p, w, d)

===== CBC =====
Status:  0
Número de Variáveis:  120
Número de Restrições:  225
Tempo de execução (s): 69.6430
Nós B&B:  164140
Função-objetivo:  57657.0
Best Bound: 57657.0000
GAP: 0.0000
Lista de Alocação (tarefa, início):
[(0, 0.0), (12, 53.0), (4, 106.0), (10, 243.0), (3, 390.0), (5, 445.0), (1, 518.0), (6, 659.0), (2, 787.0), (7, 866.0), (9, 920.0), (13, 985.0), (8, 1035.0), (11, 1162.0), (14, 1244.0)]



In [ ]:
# *********************** 15 TAREFAS CP-SAT ***************************
# Leitura do arquivo com os dados
# N (qtde tarefas), p (proc), w (peso), d (prazo) são as saídas da função leitura
N, p, w, d = leitura("instancia15.txt")  

print("===== CP-SAT =====")
modelo_Min_TerminoPonderado_CPSAT(N, p, w, d)

===== CP-SAT =====
Status:  OPTIMAL
Função-objetivo:  57657.0
Best Bound: 57657.0000
Nós B&B:  2430
GAP: 0.0000
CpSolverResponse:
status: OPTIMAL
objective: 57657
best_bound: 57657
booleans: 108
conflicts: 1597
branches: 2430
propagations: 40042
integer_propagations: 59572
walltime: 0.301298
usertime: 0.301299
deterministic_time: 0.0821814
primal_integral: 0.0862926

Lista de Alocação (tarefa, início):
[(0, 0), (12, 53), (4, 106), (10, 243), (3, 390), (5, 445), (1, 518), (6, 659), (2, 787), (7, 866), (9, 920), (13, 985), (8, 1035), (11, 1162), (14, 1244)]



In [ ]:
# *********************** 50 TAREFAS CPLEX ***************************
# Leitura do arquivo com os dados
# N (qtde tarefas), p (proc), w (peso), d (prazo) são as saídas da função leitura
N, p, w, d = leitura("instancia50.txt")  

print("===== CPLEX =====")
modelo_Min_TerminoPonderado_CPLEX(N, p, w, d)

===== CPLEX =====
Status:  time limit exceeded
Número de Variáveis:  1275
Número de Restrições:  2500
Tempo de execução (s): 3600.0299
Nós B&B:  529294
Função-objetivo:  400963
Best Bound: 110379.0794
GAP: 0.7247
Lista de Alocação (tarefa, início):
[(0, 0), (17, 53.0), (37, 106.0), (28, 163.0), (13, 231.0), (11, 281.0), (2, 363.0), (14, 442.0), (45, 562.0), (47, 684.0), (18, 808.0), (42, 859.0), (4, 927.0), (27, 1064.0), (43, 1191.0), (10, 1338.0), (34, 1485.0), (22, 1635.0), (23, 1771.0), (7, 1841.0), (24, 1895.0), (46, 2043.0), (29, 2193.0), (21, 2320.0), (9, 2406.0), (40, 2471.0), (12, 2593.0), (38, 2646.0), (33, 2785.0), (5, 2844.0), (8, 2917.0), (20, 3044.0), (16, 3176.0), (26, 3309.0), (1, 3443.0), (44, 3584.0), (41, 3730.0), (31, 3781.0), (48, 3884.0), (25, 3984.0), (19, 4088.0), (30, 4142.0), (32, 4256.0), (15, 4315.0), (6, 4383.0), (49, 4511.0), (39, 4646.0), (3, 4731.0), (35, 4786.0), (36, 4907.0)]



In [ ]:
# *********************** 50 TAREFAS CBC ***************************
# Leitura do arquivo com os dados
# N (qtde tarefas), p (proc), w (peso), d (prazo) são as saídas da função leitura
N, p, w, d = leitura("instancia50.txt")  

print("===== CBC =====")
modelo_Min_TerminoPonderado_CBC(N, p, w, d)

===== CBC =====
Status:  6
Número de Variáveis:  1275
Número de Restrições:  2500
Tempo de execução (s): 3605.2050
Nós B&B:  139676
Função-objetivo:  0.0
Best Bound: inf
GAP: -1.0000
Lista de Alocação (tarefa, início):
[(0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0), (4, 0.0), (5, 0.0), (6, 0.0), (7, 0.0), (8, 0.0), (9, 0.0), (10, 0.0), (11, 0.0), (12, 0.0), (13, 0.0), (14, 0.0), (15, 0.0), (16, 0.0), (17, 0.0), (18, 0.0), (19, 0.0), (20, 0.0), (21, 0.0), (22, 0.0), (23, 0.0), (24, 0.0), (25, 0.0), (26, 0.0), (27, 0.0), (28, 0.0), (29, 0.0), (30, 0.0), (31, 0.0), (32, 0.0), (33, 0.0), (34, 0.0), (35, 0.0), (36, 0.0), (37, 0.0), (38, 0.0), (39, 0.0), (40, 0.0), (41, 0.0), (42, 0.0), (43, 0.0), (44, 0.0), (45, 0.0), (46, 0.0), (47, 0.0), (48, 0.0), (49, 0.0)]



In [ ]:
# *********************** 50 TAREFAS CP-SAT ***************************
# Leitura do arquivo com os dados
# N (qtde tarefas), p (proc), w (peso), d (prazo) são as saídas da função leitura
N, p, w, d = leitura("instancia50.txt")  

print("===== CP-SAT =====")
modelo_Min_TerminoPonderado_CPSAT(N, p, w, d)

===== CP-SAT =====
Status:  FEASIBLE
Função-objetivo:  402764.0
Best Bound: 56193.0000
Nós B&B:  17736950
GAP: 0.8605
CpSolverResponse:
status: FEASIBLE
objective: 402764
best_bound: 56193
booleans: 2829
conflicts: 13583154
branches: 17736950
propagations: 951665242
integer_propagations: 942770689
walltime: 3600.58
usertime: 3600.58
deterministic_time: 5538.06
primal_integral: 70643.3

Lista de Alocação (tarefa, início):
[(0, 0), (17, 53), (37, 106), (28, 163), (13, 231), (11, 281), (2, 363), (14, 442), (45, 562), (47, 684), (18, 808), (42, 859), (4, 927), (27, 1064), (43, 1191), (10, 1338), (34, 1485), (35, 1635), (23, 1756), (7, 1826), (24, 1880), (46, 2028), (29, 2178), (21, 2305), (9, 2391), (40, 2456), (12, 2578), (38, 2631), (33, 2770), (5, 2829), (8, 2902), (20, 3029), (16, 3161), (26, 3294), (1, 3428), (44, 3569), (41, 3715), (31, 3766), (48, 3869), (25, 3969), (19, 4073), (30, 4127), (32, 4241), (15, 4300), (6, 4368), (49, 4496), (3, 4631), (39, 4686), (22, 4771), (36, 4907)]
